Runtime analysis of LECE and LECD for Section 4.2.5

In [ ]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.helpers.load_save_data import load_experiment_data
from src.CONFIG import DATASETS

import pandas as pd
from src.experiments.lece import NeighborhoodCalibrator
from src.experiments.temp_scaling import TemperatureScaling
from src.helpers.vector_functions import normalize_by_sum, minkowski_dist, kullback_leibler, uniform_weights_for_proportionally_k_closest

In [ ]:
!pip install line_profiler
from line_profiler import LineProfiler

In [ ]:
def print_avg_runtimes(fun, arg, n_seeds=1):
    lp = LineProfiler()
    lp_wrapper = lp(fun)

    for _ in range(n_seeds):
        lp_wrapper(arg)

    print(f"Runtime of {n_seeds} runs")
    lp.print_stats()

# LECE

In [ ]:
for dataset in DATASETS:
    print(dataset)
    data_uncal = load_experiment_data(dataset, "uncalibrated")
    best_params = load_experiment_data(dataset, "LECE_KL").cv_best_hyperparams

    calibrator = NeighborhoodCalibrator(**best_params)
    calibrator.fit(data_uncal.cal_p_train, data_uncal.y_train)
   
    print_avg_runtimes(calibrator.predict, data_uncal.cal_p_test)

# LECD

In [ ]:
for dataset in DATASETS:
    print(dataset)
    data_uncal = load_experiment_data(dataset, "uncalibrated")
    best_params = load_experiment_data(dataset, "LECD_KL").cv_best_hyperparams

    calibrator = NeighborhoodCalibrator(**best_params)
    calibrator.fit(data_uncal.cal_p_train, data_uncal.y_train)
   
    print_avg_runtimes(calibrator.predict, data_uncal.cal_p_test)

# TS+LECE

In [ ]:
for dataset in DATASETS:
    print(dataset)
    data_uncal_logits = load_experiment_data(dataset, "uncalibrated_logits")
    best_params = load_experiment_data(dataset, "TS_LECE_KL").cv_best_hyperparams

    def compositional_predict(logits_test):
        calibrator1 = TemperatureScaling()
        calibrator1.fit(data_uncal_logits.cal_p_train, data_uncal_logits.y_train)
        p_train = calibrator1.predict(data_uncal_logits.cal_p_train)
        p_test = calibrator1.predict(data_uncal_logits.cal_p_test)

        calibrator2 = NeighborhoodCalibrator(**best_params)
        calibrator2.fit(p_train, data_uncal_logits.y_train)

        return calibrator2.predict(p_test)
    
    print_avg_runtimes(compositional_predict, data_uncal_logits.cal_p_test)

# TS+LECD

In [ ]:
for dataset in DATASETS:
    print(dataset)
    data_uncal_logits = load_experiment_data(dataset, "uncalibrated_logits")
    best_params = load_experiment_data(dataset, "TS_LECD_KL").cv_best_hyperparams

    def compositional_predict(logits_test):
        calibrator1 = TemperatureScaling()
        calibrator1.fit(data_uncal_logits.cal_p_train, data_uncal_logits.y_train)
        p_train = calibrator1.predict(data_uncal_logits.cal_p_train)
        p_test = calibrator1.predict(data_uncal_logits.cal_p_test)

        calibrator2 = NeighborhoodCalibrator(**best_params)
        calibrator2.fit(p_train, data_uncal_logits.y_train)

        return calibrator2.predict(p_test)
    
    print_avg_runtimes(compositional_predict, data_uncal_logits.cal_p_test)